---
execute: 
  echo: false
  panel: sidebar
---

```{ojs}
d3 = require("d3@7")

// carregando dados
dados = await FileAttachment("dados/dados2/erros.csv").csv()
tabua_combinada = await FileAttachment("dados/dados1/previsoes_combinado.csv").csv()

// filtros para gráficos
// região
regioes = ["Brasil", "Norte", "Nordeste", "Centro-Oeste", "Sudeste", "Sul", "Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará", "Distrito Federal", "Espírito Santo", "Goiás", "Maranhão", "Mato Grosso", "Mato Grosso do Sul", "Minas Gerais", "Pará", "Paraíba", "Paraná", "Pernambuco", "Piauí", "Rio de Janeiro", "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia", "Roraima", "Santa Catarina", "São Paulo", "Sergipe", "Tocantins"]
viewof regiao = Inputs.select(regioes.sort(),
  {
        value: ['Brasil'],
        label: "Local: "
    })

// faixa etária
faixa_etaria_num =[0, 1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
faixas_etaria = ['0', '1-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90+']
viewof faixa = Inputs.select(faixas_etaria.sort(),
  {
        value: ['0'],
        label: "Faixa Etária: "
    })

// modelo
viewof modelo = Inputs.checkbox(['Lee-Carter', 'Lee-Miler'],
    {
        value: ['Lee-Miler', 'Lee-Carter'],
        label: 'Modelos:'
    }
)
modelo
```

In [ ]:
import pandas as pd

dados = pd.read_csv("dados/dados2/erros.csv")
print(dados.head())

```{ojs}
{
  const width = 760;
  const height = 500;
  const margin = { top: 60, right: 30, bottom: 50, left: 50 };
  const cores = {
    mx_lc: "steelblue",
    mx_lm: "darkorange"
  };

  const svg = d3.create("svg")
    .attr("width", width)
    .attr("height", height);

  // Título principal
  svg.append("text")
    .attr("x", width / 2)
    .attr("y", 30)
    .attr("text-anchor", "middle")
    .text("Mortalidade por Faixa Etária no Brasil (2000 - 2023)")
    .attr("font-size", "16px")
    .attr("font-weight", "bold");

  // Legenda
  const legend = svg.append("g")
    .attr("transform", `translate(${width/2 - 120}, 50)`);

  Object.entries(cores).forEach(([key, color], i) => {
    legend.append("rect")
      .attr("x", i * 100)
      .attr("y", 0)
      .attr("width", 15)
      .attr("height", 15)
      .attr("fill", color);

    legend.append("text")
      .attr("x", i * 100 + 20)
      .attr("y", 12)
      .text(key.charAt(0).toUpperCase() + key.slice(1))
      .attr("font-size", "12px");
  });

  // Processamento dos dados para todo o Brasil
  const filteredData = dados
    .filter(d => d.regiao === regiao && d.faixa_etaria === faixa && d.taxa === 'Total')
    .map(d => ({
      ano: +d.ano,
      mx_lm: Math.log(+d.mx_lm),
      mx_lc: Math.log(+d.mx_lc) // Adiciona transformação
      })).sort((a, b) => a.ano - b.ano);

  // Escalas únicas para todo o gráfico
  const xScale = d3.scaleLinear()
    .domain(d3.extent(filteredData, d => d.ano))
    .range([margin.left, width - margin.right]);

  const yValues = filteredData.flatMap(d => [d.mx_lm, d.mx_lc]);
  const yScale = d3.scaleLinear()
    .domain([d3.min(yValues), d3.max(yValues)]).nice()
    .range([height - margin.bottom, margin.top]);

  // Eixos
  svg.append("g")
    .attr("transform", `translate(0, ${height - margin.bottom})`)
    .call(d3.axisBottom(xScale).ticks(10))
    .append("text")
    .attr("x", width / 2)
    .attr("y", 35)
    .attr("text-anchor", "middle")
    .text("Ano");

  svg.append("g")
    .attr("transform", `translate(${margin.left}, 0)`)
    .call(d3.axisLeft(yScale).ticks(8))
    .append("text")
    .attr("transform", "rotate(-90)")
    .attr("y", -40)
    .attr("x", -height / 2)
    .attr("text-anchor", "middle")
    .text("log(Mx)");

  // Função para gerar as linhas
  const gerarLinha = (variavel, cor) => {
    const linha = d3.line()
      .x(d => xScale(d.ano))
      .y(d => yScale(d[variavel]))
      //.curve(d3.curveMonotoneX); //suaviza a linha

    svg.append("path")
      .datum(filteredData)
      .attr("d", linha)
      .attr("fill", "none")
      .attr("stroke", cor)
      .attr("stroke-width", 2);
  };

  // Adiciona as três linhas principais
  gerarLinha("mx_lm", cores.mx_lc);
  gerarLinha("mx_lc", cores.mx_lm);

  return svg.node();
}
```

```{ojs}
anos = Array.from({length: 2023 - 2000 + 1}, (_, i) => 2000 + i)
viewof ano = Inputs.select(anos.sort(),
  {
        value: ['2000'],
        label: "Ano: "
    })
{
  const width = 760;
  const height = 500;
  const margin = { top: 60, right: 30, bottom: 120, left: 50 }; // Aumentei a margem inferior
  const cores = {
    mx_lc: "steelblue",
    mx_lm: "darkorange"
  };

  // Ordem das faixas etárias
  const faixaEtariaOrder = [
    '0', '1-4', '5-9', '10-14', '15-19', '20-24', '25-29',
    '30-34', '35-39', '40-44', '45-49', '50-54', '55-59',
    '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90+'
  ];

  const svg = d3.create("svg")
    .attr("width", width)
    .attr("height", height);

  // Título (mantido)
  svg.append("text")
    .attr("x", width / 2)
    .attr("y", 30)
    .attr("text-anchor", "middle")
    .text("Mortalidade por Faixa Etária no Brasil (2000 - 2023)")
    .attr("font-size", "16px")
    .attr("font-weight", "bold");

  // Legenda (mantida)
  const legend = svg.append("g")
    .attr("transform", `translate(${width/2 - 120}, 50)`);

  Object.entries(cores).forEach(([key, color], i) => {
    legend.append("rect")
      .attr("x", i * 100)
      .attr("y", 0)
      .attr("width", 15)
      .attr("height", 15)
      .attr("fill", color);

    legend.append("text")
      .attr("x", i * 100 + 20)
      .attr("y", 12)
      .text(key.charAt(0).toUpperCase() + key.slice(1))
      .attr("font-size", "12px");
  });

  // Processamento dos dados agrupado por faixa etária
  const groupedData = d3.rollup(
    dados.filter(d => d.regiao === regiao && d.taxa === 'Total' && d.ano == ano),
    v => ({
      mx_lm: d3.mean(v, d => Math.log(+d.mx_lm)),
      mx_lc: d3.mean(v, d => Math.log(+d.mx_lc))
    }),
    d => d.faixa_etaria
  );

  const filteredData = faixaEtariaOrder.map(faixa => ({
    faixa_etaria: faixa,
    ...(groupedData.get(faixa) || { mx_lm: 0, mx_lc: 0 })
  }));

  // Escalas
  const xScale = d3.scalePoint()
    .domain(faixaEtariaOrder)
    .range([margin.left, width - margin.right])
    .padding(0.5);

  const yValues = filteredData.flatMap(d => [d.mx_lm, d.mx_lc]);
  const yScale = d3.scaleLinear()
    .domain([d3.min(yValues), d3.max(yValues)]).nice()
    .range([height - margin.bottom, margin.top]);

  // Eixo X com rotação
  const xAxis = svg.append("g")
    .attr("transform", `translate(0, ${height - margin.bottom})`)
    .call(d3.axisBottom(xScale));

  xAxis.selectAll("text")
    .attr("transform", "rotate(-45)")
    .attr("text-anchor", "end")
    .attr("dx", "-0.5em")
    .attr("dy", "0.15em");

  xAxis.append("text")
    .attr("x", width / 2)
    .attr("y", 60) // Ajuste para posicionar o label
    .attr("text-anchor", "middle")
    .text("Faixa Etária");

  // Eixo Y (mantido)
  svg.append("g")
    .attr("transform", `translate(${margin.left}, 0)`)
    .call(d3.axisLeft(yScale).ticks(8))
    .append("text")
    .attr("transform", "rotate(-90)")
    .attr("y", -40)
    .attr("x", -height / 2)
    .attr("text-anchor", "middle")
    .text("log(Mx)");

  // Função para gerar as linhas
  const gerarLinha = (variavel, cor) => {
    const linha = d3.line()
      .x(d => xScale(d.faixa_etaria))
      .y(d => yScale(d[variavel]));

    svg.append("path")
      .datum(filteredData)
      .attr("d", linha)
      .attr("fill", "none")
      .attr("stroke", cor)
      .attr("stroke-width", 2);
  };

  // Adiciona as linhas
  gerarLinha("mx_lm", cores.mx_lc);
  gerarLinha("mx_lc", cores.mx_lm);

  return svg.node();
}
```